<a href="https://colab.research.google.com/github/danielsreis/Challenge-Ciencia-de-dados-Aluara-2ed-do-Daniel/blob/main/Challenge_Alura_2_ed_do_Daniel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando pacotes opicionais

In [ ]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

print(' ')
print('Instalação concluída._____________________________________________________________________________________')
print(' ')

 
Instalação concluída._____________________________________________________________________________________
 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

# Instalando Pyspark - metodo simples


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 19 kB/s 
     |████████████████████████████████| 199 kB 39.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=147b82b145a2d7cb068d1d98d43002ad3b266ca46d68ddf0ebc802eeadeef6e3
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


# Instalação comum

In [ ]:
 # Importando o SparkSession
from pyspark.sql import SparkSession
# Criando a seção Spark
spark = spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate() 
# Verificando as informações da seção do Spark
spark



In [ ]:
from pyspark.sql.functions import explode
import numpy as np  
import pandas as pd

# Carregando o arquivo

In [ ]:
%%time
df = spark.read.json('dataset_bruto.json')


CPU times: user 104 ms, sys: 12.3 ms, total: 117 ms
Wall time: 14.4 s


In [ ]:
%%time
df_pandas = pd.read_json('dataset_bruto.json')

CPU times: user 3.99 s, sys: 1.62 s, total: 5.62 s
Wall time: 5.65 s


In [ ]:
df.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

# Exploração

In [ ]:
df.count()

89083

In [ ]:
# Deletar as colunas imagens e usuário,  que não serão usadas nessa análise
colunas = ['imagens' ,  'usuario']
df.drop(*colunas).printSchema() # Tem que colocar a '*' em caso de multiplas colunas


root
 |-- _corrupt_record: string (nullable = true)



Criando uma nova varável, porque com df não apaga os usb níveis indesejáveis

In [ ]:
anuncio = df.select('anuncio.*')
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [ ]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

Planificando as colunas

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

def read_nested_json(anuncio):
    column_list = []

    for column_name in anuncio.schema.names:
        print("Outside isinstance loop: " + column_name)
        # Checking column type is ArrayType
        if isinstance(anuncio.schema[column_name].dataType, ArrayType):
            print("Inside isinstance loop of ArrayType: " + column_name)
            anuncio = anuncio.withColumn(column_name, explode_outer(column_name).alias(column_name))
            column_list.append(column_name)

        elif isinstance(anuncio.schema[column_name].dataType, StructType):
            print("Inside isinstance loop of StructType: " + column_name)
            for field in anuncio.schema[column_name].dataType.fields:
                column_list.append(col(column_name + "." + field.name).alias(column_name + "_" + field.name))
        else:
            column_list.append(column_name)

    # Selecting columns using column_list from dataframe: anuncio
    anuncio = anuncio.select(column_list)
    return anuncio

read_nested_json_flag = True

while read_nested_json_flag:
  print("Reading Nested JSON File ... ")
  anuncio = read_nested_json(anuncio)
  anuncio.show(10, False)
  read_nested_json_flag = False

  for column_name in anuncio.schema.names:
    if isinstance(anuncio.schema[column_name].dataType, ArrayType):
      read_nested_json_flag = True
    elif isinstance(anuncio.schema[column_name].dataType, StructType):
      read_nested_json_flag = True


Reading Nested JSON File ... 
Outside isinstance loop: andar
Outside isinstance loop: area_total
Inside isinstance loop of ArrayType: area_total
Outside isinstance loop: area_util
Inside isinstance loop of ArrayType: area_util
Outside isinstance loop: banheiros
Inside isinstance loop of ArrayType: banheiros
Outside isinstance loop: caracteristicas
Inside isinstance loop of ArrayType: caracteristicas
Outside isinstance loop: endereco
Inside isinstance loop of StructType: endereco
Outside isinstance loop: id
Outside isinstance loop: quartos
Inside isinstance loop of ArrayType: quartos
Outside isinstance loop: suites
Inside isinstance loop of ArrayType: suites
Outside isinstance loop: tipo_anuncio
Outside isinstance loop: tipo_unidade
Outside isinstance loop: tipo_uso
Outside isinstance loop: vaga
Inside isinstance loop of ArrayType: vaga
Outside isinstance loop: valores
Inside isinstance loop of ArrayType: valores
+-----+----------+---------+---------+------------------+---------------+-

In [ ]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- endereco_bairro: string (nullable = true)
 |-- endereco_cep: string (nullable = true)
 |-- endereco_cidade: string (nullable = true)
 |-- endereco_estado: string (nullable = true)
 |-- endereco_latitude: double (nullable = true)
 |-- endereco_longitude: double (nullable = true)
 |-- endereco_pais: string (nullable = true)
 |-- endereco_rua: string (nullable = true)
 |-- endereco_zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores_condominio: string (nullable = true)
 |-- valores_iptu: string (nullable = true)
 |-- valor

In [ ]:
anuncio.show()

+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------+------------+-------------+
|andar|area_total|area_util|banheiros|   caracteristicas|endereco_bairro|endereco_cep|endereco_cidade|endereco_estado|endereco_latitude|endereco_longitude|endereco_pais|        endereco_rua|endereco_zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valores_condominio|valores_iptu|valores_tipo|valores_valor|
+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------

Mudando o tipo das colunas

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType,IntegerType, FloatType

cols_to_cast = [ "andar", "area_total" , 'area_util', 'banheiros', 'endereco_cep', 'quartos', 'suites', 'vaga', 'valores_condominio', 'valores_iptu',  'valores_valor']
anuncio = anuncio.select([F.col(c).cast('int') if c in cols_to_cast else c for c in anuncio.columns])

anuncio.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- endereco_bairro: string (nullable = true)
 |-- endereco_cep: integer (nullable = true)
 |-- endereco_cidade: string (nullable = true)
 |-- endereco_estado: string (nullable = true)
 |-- endereco_latitude: double (nullable = true)
 |-- endereco_longitude: double (nullable = true)
 |-- endereco_pais: string (nullable = true)
 |-- endereco_rua: string (nullable = true)
 |-- endereco_zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores_condominio: integer (nullable = true)
 |-- valores_iptu: integer (nullabl

# Filtros

In [ ]:
# Mostrar apenas bairro e zona 
endereço = anuncio[['endereco_bairro', 'endereco_zona' ]].distinct()
endereço.show()

+--------------------+-------------+
|     endereco_bairro|endereco_zona|
+--------------------+-------------+
|           Madureira|   Zona Norte|
|          Cavalcanti|   Zona Norte|
|              Catete|   Zona Oeste|
|Recreio dos Bande...|   Zona Norte|
|            Flamengo|             |
|     Parada de Lucas|     Zona Sul|
|   Quintino Bocaiúva|   Zona Norte|
|        Campo Grande|   Zona Oeste|
|   Engenho da Rainha|   Zona Norte|
|              Gamboa|     Zona Sul|
|             Benfica|   Zona Norte|
|        Santa Teresa| Zona Central|
|    Parque da Matriz|             |
|               Bangu|   Zona Oeste|
|         São Conrado|   Zona Oeste|
|               Lagoa|             |
|              Catete|     Zona Sul|
|            Curicica|     Zona Sul|
|         Jacarepaguá|   Zona Oeste|
|             Deodoro|   Zona Oeste|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
venda = anuncio.filter(anuncio.valores_tipo == 'Venda')
venda.show()

+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------+------------+-------------+
|andar|area_total|area_util|banheiros|   caracteristicas|endereco_bairro|endereco_cep|endereco_cidade|endereco_estado|endereco_latitude|endereco_longitude|endereco_pais|        endereco_rua|endereco_zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valores_condominio|valores_iptu|valores_tipo|valores_valor|
+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------

In [ ]:
'''

* tipo_uso: **Residencial**;
* tipo_unidade: **Apartamento**;
* tipo_anuncio: **Usado**.
'''
filtro = anuncio.filter( (anuncio.tipo_anuncio == 'Usado') & (anuncio.tipo_unidade == 'Apartamento' )&  ( anuncio.tipo_uso == 'Residencial' )).show()


+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------+------------+-------------+
|andar|area_total|area_util|banheiros|   caracteristicas|endereco_bairro|endereco_cep|endereco_cidade|endereco_estado|endereco_latitude|endereco_longitude|endereco_pais|        endereco_rua|endereco_zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valores_condominio|valores_iptu|valores_tipo|valores_valor|
+-----+----------+---------+---------+------------------+---------------+------------+---------------+---------------+-----------------+------------------+-------------+--------------------+-------------+--------------------+-------+------+------------+------------+-----------+----+------------------+------------

In [ ]:
anuncio.describe().show()

+-------+------------------+------------------+-----------------+------------------+---------------+--------------------+------------------+------------------+-----------------+-------------------+------------------+-------------+------------+-------------+--------------------+------------------+------------------+------------+------------+-----------+------------------+------------------+-----------------+------------+------------------+
|summary|             andar|        area_total|        area_util|         banheiros|caracteristicas|     endereco_bairro|      endereco_cep|   endereco_cidade|  endereco_estado|  endereco_latitude|endereco_longitude|endereco_pais|endereco_rua|endereco_zona|                  id|           quartos|            suites|tipo_anuncio|tipo_unidade|   tipo_uso|              vaga|valores_condominio|     valores_iptu|valores_tipo|     valores_valor|
+-------+------------------+------------------+-----------------+------------------+---------------+--------------

# Salvando em Parquet

In [ ]:
anuncio.write.parquet("anuncio.parquet")

Espaço utilizado 5 KB

# Salvando em csv

In [ ]:
anuncio.write.csv('anuncio_csv.csv')

Espaço utilizado 66 KB